# Cartopy 和 GeoViews
要求：
- 保留 Markdown 格式
- 保留代码、URL 和 HTML 标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

翻译：
Satpy 的目标之一是使其生成的数据能够被其他开源 Python 工具使用。对于 Python 开发者来说，Cartopy 和 GeoViews 是两个出色的绘图选项。在本课中，我们将逐步演示一些简单的示例，使用这些工具绘制数据。
首先，让我们创建我们的CONUS扇区ABI `Scene`并加载我们想要查看的一个波段。

In [ ]:
%run ../init_notebook.py
from glob import glob
from satpy import Scene
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)

In [ ]:
my_channel = 'EDITME'
scn.load([my_channel])

与之前的课程类似，我们使用笔记本的魔法命令 `%matplotlib` 来在我们的笔记本中生成交互式的 matplotlib 图表。总体而言，我们将使用 matplotlib 库提供的绘图功能，cartopy 处理海岸线和地理坐标，以及 xarray 在绘制数据时的一些实用工具。Satpy（更准确地说是 Pyresample）带来的主要功能是 `to_cartopy_crs` 方法，用于将我们的 `AreaDefinition` 转换为 cartopy 可理解的坐标参考系统（CRS）对象。
一旦我们有了那个 `crs` 对象，就可以将其传递给轴对象创建，以告知它我们绘图所使用的坐标系。我们也将这个 `crs` 传递给 `imshow` 方法，以告知它数据所处的坐标系。这与我们在重采样课程中遇到的情况类似。我们有一个想要的投影（我们的轴对象）和数据所在的投影。在这种情况下，它们是相同的，因此不需要进行重采样。
我们还将使用 cartopy 的 `海岸线` 和 `网格线` 来为我们的地图添加一些要素。

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

crs = scn[my_channel].attrs['area'].to_cartopy_crs()
plt.figure()
ax = plt.axes(projection=crs)

my_data = scn[my_channel]
my_data.plot.imshow(transform=crs)
ax.coastlines()
ax.gridlines()

如果我们想将数据绘制在不同于当前投影的其他投影上，可以让 cartopy 为我们进行一些重采样。通常卫星数据涉及相当大的数据数组，因此从长远来看，使用 Satpy 的重采样方法可能更高效，并且允许缓存重采样计算结果。
让我们绘制与之前相同的数据，但这次将其放在一个纬度/经度网格（PlateCarree）上。我们将让cartopy为我们进行重采样。

In [ ]:
import cartopy.crs as ccrs
crs = scn[my_channel].attrs['area'].to_cartopy_crs()
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

my_data = scn[my_channel]
my_data.plot.imshow(transform=crs)
ax.coastlines()
ax.gridlines()

这涵盖了使用 Cartopy 绘制 Satpy 数据的基础内容。Cartopy 提供了比此处展示的更多功能，网上有许多教程和示例，可用于制作更高级的图表。
### 进一步研究
* [Xarray 绘图][1]* [Cartopy 教程 by Phil Elson][2]* [Cartopy 文档][3]
[1]: http://xarray.pydata.org/en/stable/plotting.html[2]: https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html[3]: https://scitools.org.uk/cartopy/docs/latest/  

# 地理视图
另一个流行的绘图工具是 PyViz 团队开发的 GeoViews。Satpy 提供了一些简单的封装方法，可以轻松地从 Satpy 的 `Scene` 生成 GeoViews 图表。让我们从之前创建的 `scn` 对象开始进行一些基础绘图。首先，我们需要导入 geoviews 并告知其加载用于绘制数据的 bokeh 扩展。

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts
gv.extension("bokeh")
opts.defaults(
    opts.Image(width=600, height=400, colorbar=True),
    opts.Feature(apply_ranges=False),
    opts.QuadMesh(width=600, height=400, colorbar=True),
)

然后我们需要使用一些笔记本的魔法命令来定义我们的绘图区域。
为了使我们的数据适用于 GeoViews，我们必须使用 `Scene` 对象的 `to_geoviews` 方法，并告知其所需的通道。然后，我们可以使用乘法运算符向图表中添加诸如海岸线和国界线等要素。

In [ ]:
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r')
gv_data * gf.coastline * gf.borders

默认情况下，GeoViews 会猜测最适合查看我们数据的投影。我们也可以像使用 cartopy 一样，通过 cartopy 坐标参考系统对象来更改投影。让我们以数据的原始投影查看数据。

In [ ]:
crs = scn[my_channel].attrs['area'].to_cartopy_crs()
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r', projection=crs)
gv_data * gf.coastline * gf.borders

- 或者我们可以尝试使用兰伯特等角圆锥投影：

In [ ]:
import cartopy.crs as ccrs
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r', projection=ccrs.LambertConformal())
gv_data * gf.coastline * gf.borders

## 时间序列
最后，我们可以利用GeoViews可视化多个“数据帧”的能力。我们可以结合Satpy的`MultiScene`，轻松地逐步查看不同时间点的数据。让我们收集一小时内每10分钟的文件。

In [ ]:
from glob import glob
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/*s201813121[012345]0*.nc')
len(filenames)

In [ ]:
from satpy import MultiScene
mscn = MultiScene.from_files(filenames, reader='abi_l1b')
mscn.load(['C07'])

我们已经创建了包含多个帧的 MultiScene，并加载了想要可视化的数据产品。将这些“帧”导入 GeoViews 的第一步是再次使用 `MultiScene.blend` 方法，但与上一课中将图像堆叠在一起的做法不同，这次我们将构建一个时间序列数据数组。这类似于我们之前处理的普通二维数据数组，但增加了一个额外的 `time` 维度。

In [ ]:
from satpy.multiscene import timeseries
ts_scn = mscn.blend(timeseries)
ts_scn['C07']

In [ ]:
ts_gv = ts_scn.to_geoviews(vdims=['C07'], dynamic=True)
ts_gv.opts(cmap='viridis', projection=ccrs.Geostationary(central_longitude=-75.0))
ts_gv * gf.coastline * gf.borders

多亏了GeoViews，我们创建了一个交互式工具，用于分析一系列卫星图像，仅用了大约10行代码。